# Process timeseries of data from all satellites

Extract minimum brightness temperatures for each sensor timestep / granule, here we process: 

    - ABI (M15 @ 11.19μm)
    - AGRI (M15 @ 12.07μm)
    - AHI (M15 @ 11.23μm)
    - MODIS (B31 @ 11.03μm)
    - SLSTR (S9 @ 10.85μm)
    - VIIRS I-band (I5 @ 11.45μm)
    - VIIRS M-band (M15 @ 10.76μm)
    - AVHRR netCDF format from EUMETSAT / MetOp (B4 @ 10.8μm)
    - AVHRR GAC format from NOAA Class (B4 @ 10.8μm)

These bands were selected as being the closest to the atmospheric window. For AGRI, this is challenging as both C11 and C12 contain some contamination. C12 was chosen as it was cooler for this scene.

Each cell below (after setup cells) finds granules for a given sensor and then loops over them to find the minimum BT in a lat/lon bbox centred on the VIIRS I5 cold pixel. Results are saved as a CSV file.
    

In [ ]:
from sat_utils import do_writing
from satpy import Scene
from tqdm import tqdm
from glob import glob
import numpy as np
import warnings
import logging
import os

warnings.filterwarnings('ignore')

satpy_logger = logging.getLogger("satpy")
satpy_logger.setLevel(logging.ERROR)

In [ ]:
def dosat(infs, band, reader, storm_bbox, outfid):
    scn = Scene(infs, reader=reader)
    scn.load([band])
    ll_box = scn[band].area.area_extent_ll
    
    if np.all(np.array(ll_box)==np.inf):
        pass
    else:
        if storm_bbox[2] < ll_box[0]:
            print(ll_box, storm_bbox)
            return
        if storm_bbox[0] > ll_box[2]:
            print(ll_box, storm_bbox)
            return
        if storm_bbox[3] < ll_box[1]:
            print(ll_box, storm_bbox)
            return
        if storm_bbox[1] > ll_box[3]:
            print(ll_box, storm_bbox)
            return
    n = scn.crop(ll_bbox=storm_bbox)
    data = n[band].values
    do_writing(outfid, data, scn.attrs['start_time'].strftime("%Y%m%d%H%M%S"))
    return

def dosat_polar(infs, band, reader, storm_bbox, outfid):
    scn = Scene(infs, reader=reader)
    try:
        scn.load([band])
        lons, lats = scn[band].area.get_lonlats()
        data = scn[band].values
    except Exception as e:
        print("Uh-oh, error with:", infs)
        print(e)
        return
    
    pts = (lons < storm_bbox[0]).nonzero()
    data[pts] = np.nan
    pts = (lats < storm_bbox[1]).nonzero()
    data[pts] = np.nan
    pts = (lons > storm_bbox[2]).nonzero()
    data[pts] = np.nan
    pts = (lats > storm_bbox[3]).nonzero()
    data[pts] = np.nan
    do_writing(outfid, data, scn.attrs['start_time'].strftime("%Y%m%d%H%M%S"))
    return

In [ ]:
# First, set up a bounding box around the VIIRS minimum BT.
# We will extract each satellite's data over this bbox.
minbt_lat = -3.2609048
minbt_lon = 163.2608

storm_bbox = (minbt_lon-2, minbt_lat-2,
              minbt_lon+2, minbt_lat+2)


# For VIIRS we create a list of dates, as in some cases multiple granules
# overlap the `storm_bbox` and hence we'd have two 'minimum' values for
# granules on the same orbit.
viirs_date_list = ['*d20181227_t0128444*',
                   '*d20181227_t0305290*',
                   '*d20181227_t0311103*',
                   '*d20181227_t1411118*',
                   '*d20181227_t1416530*',
                   '*d20181228_t0105338*',
                   '*d20181228_t0111150*',
                   '*d20181228_t0247579*',
                   '*d20181228_t0253391*',
                   '*d20181228_t1353424*',
                   '*d20181229_t0230285*',
                   '*d20181229_t1330300*',
                   '*d20181229_t1336113*',
                   '*d20181229_t1341543*',
                   '*d20181229_t1512559*',
                   '*d20181229_t1518371*',
                   '*d20181230_t0212592*',
                   '*d20181230_t1313007*',
                   '*d20181230_t1318419*',
                   '*d20181230_t1455265*',
                   '*d20181231_t0149468*',
                   '*d20181231_t0155280*',
                   '*d20181231_t0332127*',
                   '*d20181231_t1255313*',
                   '*d20181231_t1301126*',
                   '*d20181231_t1437554*',
                   '*d20181231_t1341020*',
                   '*d20181231_t1346437*',
                   '*d20181231_t1352253*']

In [ ]:
# This is for AGRI
indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/AGRI/'
outfid = open('../data/sat_csv/AGRI_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')
files = glob(indir + '*.HDF')
files.sort()
count = 0
for inf in tqdm(files):
    dosat([inf], 'C12', 'agri_l1', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for AHI FD
indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/AHI_F/'
outfid = open('../data/sat_csv/AHI_TEMPS_F.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

files = glob(indir + '*B14*S0110*.DAT*')
files.sort()
count = 0
for inf in tqdm(files):
    pos = inf.find('B14')
    dtstr = inf[pos-14:pos-1]
    flist = glob(indir + '*' + dtstr + '*.DAT')
    dosat(flist, 'B14', 'ahi_hsd', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for AHI Meso
# There is no data available, so it's not worth running this. Kept for use with other features of interest.

indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/AHI_MESO/'
outfid = open('../data/sat_csv/AHI_TEMPS_MESO.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

files = glob(indir + '*B14*.DAT*')
files.sort()
count = 0
for inf in tqdm(files):
    dosat([inf], 'B14', 'ahi_hsd', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for ABI FD
indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/ABI/'
outfid = open('../data/sat_csv/ABI_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

files = glob(indir + '*C14*.nc')
files.sort()
count = 0
for inf in tqdm(files):
    dosat([inf], 'C14', 'abi_l1b', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for MODIS

indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/MODIS/'
outfid = open('../data/sat_csv/MODIS_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

infiles = glob(indir + '*D021KM.A*.hdf')

for inf in tqdm(infiles):
    try:
        dosat_polar([inf], '31', 'modis_l1b', storm_bbox, outfid)
    except ValueError:
        pass
outfid.close()

In [ ]:
# This is for VIIRS-I
indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/VIIRSI/'
outfid = open('../data/sat_csv/VIIRS_IBAND_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

for dts in tqdm(viirs_date_list):
    files = glob(indir + dts + '.h5')
    dosat_polar(files, 'I05', 'viirs_sdr', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for VIIRS-M

indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/VIIRSM/'
outfid = open('../data/sat_csv/VIIRS_MBAND_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

for dts in tqdm(viirs_date_list):
    files = glob(indir + dts + '.h5')
    if len(files) < 1:
        continue
    dosat_polar(files, 'M15', 'viirs_sdr', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for SLSTR
indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/SLSTR/'
outfid = open('../data/sat_csv/SLSTR_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

dirs = glob(indir + '*.SEN3')
dirs.sort()

for curdir in tqdm(dirs):
    files = glob(curdir + '/*.nc')
    dosat_polar(files, 'S9', 'slstr_l1b', storm_bbox, outfid)
outfid.close()

In [ ]:
# This is for AVHRR EUM NC

indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/AVHRR/EUM/'
outfid = open('../data/sat_csv/AVHRR_EUM_NC_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

infiles = glob(indir + '*.nat')

for inf in tqdm(infiles):
    try:
        dosat_polar([inf], '4', 'avhrr_l1b_eps', storm_bbox, outfid)
    except ValueError:
        pass
outfid.close()

In [ ]:
# This is for AVHRR CLASS Format
indir = '/gf2/eodg/SRP001_PROUD_TURBREP/Supercold/AVHRR/CLASS/'
outfid = open('../data/sat_csv/AVHRR_CLASS_TEMPS.csv', 'w')
outfid.write('Time,Min,1pc,2pc,5pc,10pc,25pc,50pc,90pc,Max,Mean,Median,Std\n')

infiles = glob(indir + '*GHRR*')

for inf in tqdm(infiles):
    try:
        dosat_polar([inf], '4', 'avhrr_l1b_gaclac', storm_bbox, outfid)
    except ValueError:
        pass
outfid.close()